In [57]:
import os
from json import load

In [58]:
data = '../../data/us/congress/data'
sessions = ['113', '114', '115']

In [59]:
bills = {}
sponsors = {}

In [60]:
def process_bill_json(fpath):
    cosponsorships = []
    with open(fpath, 'r') as f:
        data = load(f)
        all_sponsors = [data['sponsor']] + data['cosponsors']
        for i, sponsor in enumerate(all_sponsors):
            if sponsor['bioguide_id'] not in sponsors:
                sponsors[sponsor['bioguide_id']] = sponsor
            for cosponsor in all_sponsors[i:]:
                if cosponsor['bioguide_id'] not in sponsors:
                    sponsors[cosponsor['bioguide_id']] = cosponsor
                if sponsor['bioguide_id'] != cosponsor['bioguide_id']:
                    cosponsorships.append((sponsor['bioguide_id'], cosponsor['bioguide_id']))

    return cosponsorships

In [61]:
for session in sessions:
    path = os.path.join(data, session)
    for dir_ in os.listdir(path):
        if dir_ == 'bills':
            dir_path = os.path.join(path, dir_)
            for type_ in os.listdir(dir_path):
                type_path = os.path.join(dir_path, type_)
                with open(f'../../data/us/raw/{session}/{type_[0]}/cosponsorships.csv', 'w+') as out:
                    for dirpath, dirnames, filenames in os.walk(type_path):
                        for file in filenames:          
                            if file.endswith('.json'):
                                csps = process_bill_json(os.path.join(dirpath, file))
                                for sp, csp in csps:
                                    out.write(f'{sp},{csp}\n')

In [65]:
with open('../../data/us/raw/legislators/sponsors.csv', 'w+') as f:
    f.write('bioguide_id;name\n')
    for sponsor, info in sponsors.items():
        id_ = info['bioguide_id']
        name = info['name']
        f.write(f'{id_};{name}\n')